<a href="https://colab.research.google.com/github/santiago2588/distillation_column_training/blob/main/Soluciones_colab/05_gradient_boosting_regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Data wrangling
import pandas as pd
import numpy as np

In [2]:
# Data visualization
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
from tqdm import tqdm

In [3]:
# Models
from sklearn.ensemble import GradientBoostingRegressor

In [4]:
# Models Pipelines
from sklearn.pipeline import Pipeline

In [6]:
# Install the optuna library
!pip install optuna -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.7/242.7 kB 14.0 MB/s eta 0:00:00


In [7]:
#Hyperparameter optimization
import optuna

In [8]:
# Model evaluation
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [9]:
# Save model
import joblib

In [10]:
!wget https://raw.githubusercontent.com/santiago2588/distillation_column_training/main/data/transformed_data.csv -O transformed_data.csv

--2025-07-07 14:00:09--  https://raw.githubusercontent.com/santiago2588/distillation_column_training/main/data/transformed_data.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 12344 (12K) [text/plain]
Saving to: ‘transformed_data.csv’

transformed_data.cs 100%[===================>]  12.05K  --.-KB/s    in 0.003s  

2025-07-07 14:00:10 (4.43 MB/s) - ‘transformed_data.csv’ saved [12344/12344]



In [11]:
import pandas as pd

# Read the CSV file into a DataFrame
df = pd.read_csv("transformed_data.csv")

# Display the first few rows to confirm it loaded correctly
df.head()

,PressureC1_diff,FlowC1,Temp1,Yield
0,0.0000,432.0636,139.9857,69.400623
1,-9.9628,487.4029,131.0470,66.532666
2,-0.0695,437.3516,118.2666,71.102193
3,0.2257,481.8314,118.1769,69.793481
4,-0.1678,412.6471,120.7891,71.489516


In [12]:
# Define features (X) and target variable (y)
X = df.drop('Yield', axis=1)  # Features (all columns except 'Job Offer')
y = df['Yield']  # Target variable

In [13]:
from sklearn.model_selection import train_test_split
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
# Check the shape of the data
print(f"Training data shape: {X_train.shape}")
print(f"Testing data shape: {X_test.shape}")

Training data shape: (202, 3)
Testing data shape: (51, 3)


In [14]:
def get_metrics(y_true, y_pred):

    dict_metrics = {
        'MSE': mean_squared_error(y_true, y_pred),
        'MAE': mean_absolute_error(y_true, y_pred),
        'R2 Score': r2_score(y_true, y_pred),

    }
    return dict_metrics

In [15]:
# Creating pipeline with Boosted Trees
pip_model_bt = Pipeline(steps=[
    ('model', GradientBoostingRegressor(random_state=2023))
])

# Fit pipeline
pip_model_bt.fit(X_train, y_train)

# Generate Predictions using the correctly fitted pipeline
y_pred = pip_model_bt.predict(X_test)

# Evaluate Metrics
metrics = get_metrics(y_test, y_pred)

# View Results
metrics

{'MSE': 0.5607937224219373,
 'MAE': 0.554537030979557,
 'R2 Score': 0.7250647125960571}

In [16]:
#Optimize hyperparameters with Optuna

# Define the objective function
def objective(trial):
    # Suggest hyperparameters to tune
    n_estimators = trial.suggest_int("n_estimators", 50, 500)
    max_depth = trial.suggest_int("max_depth", 2, 10)
    learning_rate = trial.suggest_float("learning_rate", 0.01, 0.3, log=True)
    subsample = trial.suggest_float("subsample", 0.5, 1.0)
    min_samples_split = trial.suggest_int("min_samples_split", 2, 20)
    min_samples_leaf = trial.suggest_int("min_samples_leaf", 1, 20)

    # Create the pipeline with the suggested hyperparameters
    pip_model_bt = Pipeline(steps=[
        ('model', GradientBoostingRegressor(
            n_estimators=n_estimators,
            max_depth=max_depth,
            learning_rate=learning_rate,
            subsample=subsample,
            min_samples_split=min_samples_split,
            min_samples_leaf=min_samples_leaf,
            random_state=2023
        ))
    ])

    # Fit the pipeline
    pip_model_bt.fit(X_train, y_train)

    # Predict on the test set
    y_pred = pip_model_bt.predict(X_test)

    # Calculate the evaluation metric (MSE in this case)
    mse = mean_squared_error(y_test, y_pred)
    return mse  # Optuna minimizes the objective function by default

# Create a study object
study = optuna.create_study(direction="minimize")  # We want to minimize MSE

# Optimize the study
study.optimize(objective, n_trials=50)  # Run 50 trials (you can increase this for better results)

# Print the best hyperparameters
best_params = study.best_params
print("Best hyperparameters:", study.best_params)

[I 2025-07-07 14:00:34,149] A new study created in memory with name: no-name-eb9f03fc-5e64-4286-b56d-15146b099beb
[I 2025-07-07 14:00:34,525] Trial 0 finished with value: 0.6580079139516993 and parameters: {'n_estimators': 357, 'max_depth': 8, 'learning_rate': 0.23167256523522226, 'subsample': 0.7723470266271654, 'min_samples_split': 12, 'min_samples_leaf': 14}. Best is trial 0 with value: 0.6580079139516993.
[I 2025-07-07 14:00:34,953] Trial 1 finished with value: 0.44276927671751265 and parameters: {'n_estimators': 331, 'max_depth': 9, 'learning_rate': 0.05960718069034, 'subsample': 0.9114165755101982, 'min_samples_split': 7, 'min_samples_leaf': 5}. Best is trial 1 with value: 0.44276927671751265.
[I 2025-07-07 14:00:35,125] Trial 2 finished with value: 0.8982886305623583 and parameters: {'n_estimators': 185, 'max_depth': 10, 'learning_rate': 0.25702420521367036, 'subsample': 0.6574346003208865, 'min_samples_split': 10, 'min_samples_leaf': 20}. Best is trial 1 with value: 0.442769276

Best hyperparameters: {'n_estimators': 367, 'max_depth': 6, 'learning_rate': 0.09581182108365248, 'subsample': 0.863749492021017, 'min_samples_split': 9, 'min_samples_leaf': 4}


In [17]:
# Train the final model
final_model = GradientBoostingRegressor(
    n_estimators=best_params["n_estimators"],
    max_depth=best_params["max_depth"],
    learning_rate=best_params["learning_rate"],
    subsample=best_params["subsample"],
    min_samples_split=best_params["min_samples_split"],
    min_samples_leaf=best_params["min_samples_leaf"],
    random_state=2023
)

# Fit the final model
final_model.fit(X_train, y_train)

# Log metrics
y_pred = final_model.predict(X_test)
metrics = get_metrics(y_test, y_pred)

# Print metrics
print("Model Metrics:")
for metric_name, metric_value in metrics.items():
    print(f"{metric_name}: {metric_value}")

# Print best parameters
print("\nBest Hyperparameters:")
for param_name, param_value in best_params.items():
    print(f"{param_name}: {param_value}")

Model Metrics:
MSE: 0.4067005913288812
MAE: 0.5035124201958735
R2 Score: 0.8006105641100069

Best Hyperparameters:
n_estimators: 367
max_depth: 6
learning_rate: 0.09581182108365248
subsample: 0.863749492021017
min_samples_split: 9
min_samples_leaf: 4


In [18]:
# Visualize the optimization history
optuna.visualization.plot_optimization_history(study).show()

In [19]:
# Visualize the parameter importance
optuna.visualization.plot_param_importances(study).show()